In [1]:
import os

In [2]:
%pwd

'/Users/vaishalikant/Chest-Cancer-Detection/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/vaishalikant/Chest-Cancer-Detection'

In [5]:

from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list

In [6]:
from src.DLPROJECT.constants import *
from src.DLPROJECT.utils.common import read_yaml, create_directories

In [7]:

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

        

    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "Chest-CT-Scan-data")
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config

In [8]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [9]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time
from src.DLPROJECT.entity.config_entity import TrainingConfig
from pathlib import Path


class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config

    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )
        # Create a new optimizer instance
        optimizer = tf.keras.optimizers.Adam()

        # Compile the model with the new optimizer
        self.model.compile(optimizer=optimizer,
                           loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                           metrics=['accuracy'])

    def train_valid_generator(self):
        datagenerator_kwargs = dict(
            rescale=1./255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

    def train(self):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )


In [10]:
import sys
from pathlib import Path
import tensorflow as tf

# Enable eager execution
tf.config.run_functions_eagerly(True)

# Add the src directory to the Python path
sys.path.append(str(Path().resolve().parent / 'src'))

from src.DLPROJECT.config.configuration import ConfigurationManager
from src.DLPROJECT.components.model_trainer import Training
from src.DLPROJECT.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from src.DLPROJECT.entity.config_entity import TrainingConfig

try:
    # Print the current working directory and paths to verify they are correct
    print(f"Current working directory: {Path().resolve()}")
    print(f"CONFIG_FILE_PATH: {CONFIG_FILE_PATH}")
    print(f"PARAMS_FILE_PATH: {PARAMS_FILE_PATH}")

    config = ConfigurationManager(config_filepath=Path(CONFIG_FILE_PATH), params_filepath=Path(PARAMS_FILE_PATH))
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()
except FileNotFoundError as e:
    print(f"FileNotFoundError: {e}")
    raise e
except ValueError as e:
    print(f"ValueError: {e}")
    raise e
except Exception as e:
    print(f"An error occurred: {e}")
    raise e

Current working directory: /Users/vaishalikant/Chest-Cancer-Detection
CONFIG_FILE_PATH: src/DLPROJECT/configs/config.yaml
PARAMS_FILE_PATH: src/DLPROJECT/configs/params.yaml
Reading config from: src/DLPROJECT/configs/config.yaml
Reading params from: src/DLPROJECT/configs/params.yaml
[2025-03-18 17:19:59,329: INFO: common: yaml file: src/DLPROJECT/configs/config.yaml loaded successfully]
[2025-03-18 17:19:59,362: INFO: common: yaml file: src/DLPROJECT/configs/params.yaml loaded successfully]
[2025-03-18 17:19:59,363: INFO: common: created directory at: artifacts]
[2025-03-18 17:19:59,363: INFO: common: created directory at: artifacts/training]
[2025-03-18 17:19:59,476: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
Found 68 images belonging to 2 classes.
Found 275 images belonging to 2 classes.


/Users/vaishalikant/.pyenv/versions/3.9.6/lib/python3.9/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


An error occurred: numpy() is only available when eager execution is enabled.


NotImplementedError: numpy() is only available when eager execution is enabled.